In [1]:
!pip install --upgrade tables
!pip install eli5
!pip install xgboost

     |████████████████████████████████| 4.3MB 2.8MB/s 
  Found existing installation: tables 3.4.4
    Uninstalling tables-3.4.4:
      Successfully uninstalled tables-3.4.4
     |████████████████████████████████| 112kB 2.7MB/s 


In [0]:
import pandas as pd 
import numpy as np 
from sklearn.dummy import DummyRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

import xgboost as xgb
from sklearn.metrics import mean_absolute_error as mae
from sklearn.model_selection import cross_val_score, KFold
import eli5
from eli5.sklearn import PermutationImportance


In [3]:
cd "/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_two/dw_martix_car/"

/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_two/dw_martix_car


In [4]:
df = pd.read_hdf('data/car.h5')
df.shape

(106494, 155)

In [0]:
SUFFIX_CAT = '__cat'
for feat in df.columns:
    if isinstance (df[feat][0], list): continue
    
    factorize_values = df[feat].factorize()[0]
    if SUFFIX_CAT in feat:
        df[feat] = factorize_values
    else:
        df[feat +SUFFIX_CAT] = df[feat].factorize()[0]

In [6]:
cat_feats = [x for x in df.columns if SUFFIX_CAT in x]
cat_feats = [x for x in cat_feats if 'price' not in x]
len(cat_feats)

151

In [0]:
def run_model(model ,feats):
    X = df[feats].values
    y = df['price_value'].values
   
    scores = cross_val_score(model, X, y, cv=3, scoring='neg_mean_absolute_error')
    return np.mean(scores), np.std(scores)

#Decision Tree

In [12]:
run_model(DecisionTreeRegressor(max_depth=5), cat_feats)

(-19695.13091100928, 148.72570644015792)

#Random Forest

In [17]:
model = RandomForestRegressor(max_depth=5, n_estimators=50, random_state=0)
run_model(model, cat_feats )

(-18718.657185256638, 64.5424578125788)

#XGBoost

In [19]:
 xgb_params = {
     'max_depth' : 5,
     'n_estimators' : 50,
     'learning_rate' : 0.1,
     'seed': 0
 }

run_model(xgb.XGBRegressor(**xgb_params), cat_feats )

[05:38:02] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[05:38:22] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[05:38:41] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-13108.379065811214, 74.32158265003798)

In [20]:
m = xgb.XGBRegressor(max_depth=5, n_estimators=50, learning_rate=0.1, seed=0)
m.fit(X,y)
imp = PermutationImportance(m).fit(X,y)
eli5.show_weights(imp, feature_names = cat_feats)

[05:40:49] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


Weight,Feature
0.1190 ± 0.0018,param_napęd__cat
0.1120 ± 0.0024,param_rok-produkcji__cat
0.1081 ± 0.0040,param_stan__cat
0.0605 ± 0.0027,param_skrzynia-biegów__cat
0.0573 ± 0.0011,param_faktura-vat__cat
0.0482 ± 0.0018,param_moc__cat
0.0272 ± 0.0012,param_marka-pojazdu__cat
0.0239 ± 0.0010,feature_kamera-cofania__cat
0.0210 ± 0.0007,param_typ__cat
0.0176 ± 0.0018,seller_name__cat


In [0]:
feats ={'param_napęd__cat',
'param_rok-produkcji',
'param_stan__cat',
'param_skrzynia-biegów__cat',
'param_faktura-vat__cat',
'param_moc__cat',
'param_marka-pojazdu__cat',
'feature_kamera-cofania__cat',
'param_typ__cat',
'seller_name__cat',
'param_pojemność-skokowa__cat',
'feature_wspomaganie-kierownicy__cat',
'param_model-pojazdu__cat',
'param_wersja__cat',
'param_kod-silnika__cat',
'feature_system-start-stop__cat',
'feature_asystent-pasa-ruchu__cat',
'feature_czujniki-parkowania-przednie__cat',
'feature_łopatki-zmiany-biegów__cat',
'feature_regulowane-zawieszenie__cat'} 

In [25]:
run_model(xgb.XGBRegressor(**xgb_params), feats )

[05:51:51] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[05:51:55] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[05:51:59] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-11308.227280256795, 28.717500502999584)

In [0]:
df['param_rok-produkcji'] = df['param_rok-produkcji'].map(lambda x: -1 if str(x) == 'None' else int(x)) 

In [0]:

df['param_moc'] = df['param_moc'].map(lambda x: -1 if str(x) == 'None' else int(x.split(' ')[0]))

In [38]:
feats ={'param_napęd__cat','param_rok-produkcji','param_stan__cat','param_skrzynia-biegów__cat','param_faktura-vat__cat','param_moc','param_marka-pojazdu__cat','feature_kamera-cofania__cat','param_typ__cat','seller_name__cat','param_pojemność-skokowa__cat','feature_wspomaganie-kierownicy__cat','param_model-pojazdu__cat','param_wersja__cat','param_kod-silnika__cat','feature_system-start-stop__cat','feature_asystent-pasa-ruchu__cat','feature_czujniki-parkowania-przednie__cat','feature_łopatki-zmiany-biegów__cat','feature_regulowane-zawieszenie__cat'} 
run_model(xgb.XGBRegressor(**xgb_params), feats )

[06:11:41] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[06:11:45] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[06:11:49] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-9716.388060574422, 61.98569007085089)

In [0]:
df['param_pojemność-skokowa'] = df['param_pojemność-skokowa'].map(lambda x: -1 if str(x) == 'None' else int(x.split('cm')[0].replace(' ', '')))

In [39]:
feats ={'param_napęd__cat','param_rok-produkcji','param_stan__cat','param_skrzynia-biegów__cat','param_faktura-vat__cat','param_moc','param_marka-pojazdu__cat','feature_kamera-cofania__cat','param_typ__cat','seller_name__cat','param_pojemność-skokowa','feature_wspomaganie-kierownicy__cat','param_model-pojazdu__cat','param_wersja__cat','param_kod-silnika__cat','feature_system-start-stop__cat','feature_asystent-pasa-ruchu__cat','feature_czujniki-parkowania-przednie__cat','feature_łopatki-zmiany-biegów__cat','feature_regulowane-zawieszenie__cat'} 
run_model(xgb.XGBRegressor(**xgb_params), feats )

[06:14:13] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[06:14:18] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[06:14:22] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-9569.228386205847, 71.72119955520827)